# Restricted Two Body Problem: Elliptical Orbits Around a Central Mass

This is the general case of a Keperian orbit.<br>
A light body (e.g. a planet) orbits a heavy central body (e.g. the sun).  The orbit is an ellipse with the primary at one focus.

In [1]:
# Library imports
import tensorflow as tf
import rebound
import numpy as np

# Aliases
keras = tf.keras

In [2]:
# Local imports
from utils import load_vartbl, save_vartbl, plot_style
from tf_utils import gpu_grow_memory, TimeHistory
from tf_utils import plot_loss_hist, EpochLoss, TimeHistory
from tf_utils import Identity

from orbital_element import OrbitalElementToConfig, ConfigToOrbitalElement, MeanToTrueAnomaly
from orbital_element import make_model_elt_to_cfg, make_model_cfg_to_elt

from r2b_data import make_traj_r2b, make_train_r2b, make_datasets_r2b, make_datasets_earth
from r2b import KineticEnergy_R2B, PotentialEnergy_R2B, AngularMomentum_R2B, Motion_R2B
from r2b_model_math import make_position_model_r2b_math

In [3]:
# Lightweight serialization
# fname = '../data/r2b/r2bc.pickle'
# vartbl = load_vartbl(fname)

### Generate data sets and an example batch

In [4]:
# Generate one example trajectory
a = 1.0
e = 0.0
inc = 0.0
Omega = 0.0
omega = 0.0
f = 0.0
n_years = 2

inputs_traj, outputs_traj = make_traj_r2b(a=a, e=e, inc=inc, Omega=Omega, omega=omega, f=f, n_years=n_years)

In [5]:
inputs_traj.keys()

dict_keys(['t', 'q0', 'v0', 'mu'])

In [6]:
# Inputs for make_train_r2b
n_traj = 10
n_years = 2
a_min = 0.50
a_max = 32.0
e_max = 0.20
inc_max = np.pi/4.0
seed = 42

In [7]:
# Test make_train_r2b
inputs, outputs= make_train_r2b(n_traj=n_traj, n_years=n_years, a_min=a_min, a_max=a_max, 
                                e_max=e_max, inc_max=inc_max, seed=seed)

In [8]:
# Inputs for make_datasets_r2b
n_traj = 100
vt_split = 0.20
n_years = 2
a_min = 0.50
a_max = 32.0
e_max = 0.20
inc_max = np.pi/4.0
seed = 42
batch_size = 64

In [9]:
ds_trn, ds_val, ds_tst = make_datasets_r2b(n_traj=n_traj, vt_split=vt_split, n_years=n_years, a_min=a_min, a_max=a_max, 
                                e_max=e_max, inc_max=inc_max, seed=seed, batch_size=batch_size)

Loaded data from ../data/r2b/1421569704.pickle.


In [10]:
# Create DataSet objects for toy size problem - earth orbits only (a=1, e=0)
ds_earth_trn, ds_earth_val, ds_earth_tst = make_datasets_earth(n_traj=n_traj, vt_split=vt_split, n_years=n_years)

Loaded data from ../data/r2b/2367906283.pickle.


In [11]:
# Example batch
batch_in, batch_out = list(ds_earth_trn.take(1))[0]
print('Input field names: ', list(batch_in.keys()))
print('Output field names:', list(batch_out.keys()))

t = batch_in['t']
q0 = batch_in['q0']
v0 = batch_in['v0']
mu = batch_in['mu']

q = batch_out['q']
v = batch_out['v']
a = batch_out['a']
q0_rec = batch_out['q0_rec']
v0_rec = batch_out['v0_rec']
H = batch_out['H']
L = batch_out['L']

print(f'Example batch sizes:')
print(f't  = {t.shape}')
print(f'q0 = {q0.shape}')
print(f'v0 = {v0.shape}')
print(f'mu = {mu.shape}')

print(f'q  = {q.shape}')
print(f'v  = {v.shape}')
print(f'a  = {a.shape}')
# print(f'q0_rec = {q0_rec.shape}')
# print(f'v0_rec = {v0_rec.shape}')
print(f'H  = {H.shape}')
print(f'L  = {L.shape}')

Input field names:  ['t', 'q0', 'v0', 'mu']
Output field names: ['q', 'v', 'a', 'q0_rec', 'v0_rec', 'T', 'U', 'H', 'L']
Example batch sizes:
t  = (64, 731)
q0 = (64, 3)
v0 = (64, 3)
mu = (64,)
q  = (64, 731, 3)
v  = (64, 731, 3)
a  = (64, 731, 3)
H  = (64, 731)
L  = (64, 731, 3)


In [12]:
traj_size = 731

tf.debugging.assert_shapes(
    shapes = {
    # Inputs
    t: (batch_size, traj_size),
    q0: (batch_size, 3),
    v0: (batch_size, 3),
    mu: (batch_size,),
    # Outputs
    q: (batch_size, traj_size, 3),
    v: (batch_size, traj_size, 3),
    a: (batch_size, traj_size, 3),
    q0_rec: (batch_size, 3),
    v0_rec: (batch_size, 3),
    H: (batch_size, traj_size),
    L: (batch_size, traj_size, 3),
    })

**Call layers with physics computations**

In [13]:
T = KineticEnergy_R2B()(v)
T.shape

TensorShape([64, 731])

In [14]:
U = PotentialEnergy_R2B()([q, mu])
U.shape

TensorShape([64, 731])

In [15]:
L = AngularMomentum_R2B()([q, v])
L.shape

TensorShape([64, 731, 3])

**Conversion of initial configuration to orbital elements**

In [16]:
qx = q0[:,0]
qy = q0[:,1]
qz = q0[:,2]
vx = v0[:,0]
vy = v0[:,1]
vz = v0[:,2]
inputs_cart = (qx, qy, qz, vx, vy, vz, mu)

In [17]:
elts = ConfigToOrbitalElement()(inputs_cart)
a0, e0, inc0, Omega0, omega0, f0, M0, N0 = elts

In [18]:
# Review shapes
print(f'Example batch sizes:')
print(f'qx   = {qx.shape}')
print(f'vx   = {vx.shape}')
print(f'mu   = {mu.shape}')
print(f'a0   = {a0.shape}')

Example batch sizes:
qx   = (64,)
vx   = (64,)
mu   = (64,)
a0   = (64,)


### Mathematical Model
**Compute position as a function of time from initial orbital elements**

In [19]:
position_model_math = make_position_model_r2b_math(traj_size=traj_size)

In [20]:
qx, qy, qz, vx, vy, vz = position_model_math([t, q0, v0])
print(f'qx = {qx.shape}')
print(f'qy = {qy.shape}')
print(f'qz = {qz.shape}')

qx = (64, 731, 1)
qy = (64, 731, 1)
qz = (64, 731, 1)


In [21]:
# keras.utils.plot_model(position_model_math, '../model_plots/r2b/position_model_math.png')

In [22]:
# position_model_math.summary()

**Motion Model: Compute v and a from q using automatic differentiation**<br>
Factory function that accepts any position model<br>
Instantiated here from mathematical position model

In [23]:
motion_model_math = Motion_R2B(position_model=position_model_math, name='motion_model')

In [25]:
q, v, a = motion_model_math([t, q0, v0])
print('shape of motion_model outputs:')
print(f'q: {q.shape}')
print(f'v: {v.shape}')
print(f'a: {a.shape}')

shape of motion_model outputs:
q: (64, 731, 3)
v: (64, 731, 3)
a: (64, 731, 3)


In [26]:
motion_model_math.summary()

Model: "motion_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_r2b_math (Model)       ((None, 731, 1), (None, 7 0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


**Full physics model: computes energy and angular momentum from q, v**<br>
Factory function that accepts any position model.<br>
Instantiated here from mathematical position model.